In [1]:
from tensorflow.keras import datasets, layers, models

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mlp
import time

In [4]:
def padImage(image, pixels=5):
    bottom = image[-pixels:]
    top = image[:pixels]

    img = np.insert(image, 0, bottom, 0)
    img = np.insert(img, len(img), top, 0)
    img = np.insert(img, [0], [0] * pixels, 1)
    img = np.insert(img, [-1], [0] * pixels, 1)
    return img

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0).astype(np.float32), test_images.astype(np.float32) / 255.0
test = np.loadtxt('mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat')

In [6]:
### THE PAD HAS TO BE DONE IN THE
### POLAR SPACE

# 20 is the ceiling of (14 * sqrt(2))
X_train_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in train_images]
X_train_polar = [padImage(x, pixels=5) for x in X_train_polar]
X_train_polar = np.array(X_train_polar)[...,None]

X_test_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in test_images]
X_test_polar = [padImage(x, pixels=5) for x in X_test_polar]
X_test_polar = np.array(X_test_polar)[...,None]

# Rotate test set
X_test_r_polar = [tfa.image.rotate(x, np.random.uniform(-np.pi/2., np.pi/2.)).numpy() for x in test_images]
X_test_r_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in X_test_r_polar]
X_test_r_polar = [padImage(x, pixels=5) for x in X_test_r_polar]
X_test_r_polar = np.array(X_test_r_polar)[...,None]

In [181]:
#Trying Pool1D on the columns as polar is invariant horizontally, but varies on the y-axis wrt. rotations
# Columns is axis=2, because --> batch_size,row,col,filters
inputs = tf.keras.Input(shape=(38, 38, 1))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D()(x)
x = layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
x = layers.Dropout(rate=0.5)(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
x = layers.Dropout(rate=0.5)(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
#x = layers.GlobalMaxPool2D()(x) 

# global max pool on the col
x = tf.math.reduce_max(x, axis=2, keepdims=True)
x = layers.Flatten()(x)

x = layers.Dense(64, activation='linear')(x)
x = layers.Activation('relu')(x)

x = layers.Dense(10, activation='softmax')(x)

model_x = models.Model(inputs=inputs, outputs=x)

In [182]:
model_x.summary()

Model: "model_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_50 (InputLayer)        [(None, 38, 38, 1)]       0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 36, 36, 32)        320       
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
layer_normalization_54 (Laye (None, 18, 18, 32)        64        
_________________________________________________________________
dropout_54 (Dropout)         (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 8, 8, 64)          0  

In [183]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
checkpoint = tf.keras.callbacks.ModelCheckpoint(name + '.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.000, patience=5, verbose=1,
    mode='auto', baseline=None, restore_best_weights=False
)


model_x.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_x.fit(X_train_polar[:1000], train_labels[:1000], batch_size=128, epochs=10, 
          validation_data=(X_test_r_polar, test_labels),
          callbacks=[reduce_lr, checkpoint, es])

Epoch 1/10
8/8 [==============================] - 3s 394ms/step - loss: 3.1024 - accuracy: 0.1107 - val_loss: 2.3112 - val_accuracy: 0.1528

Epoch 00001: val_loss improved from inf to 2.31123, saving model to m_pol_mnist_layerAdjust.h5
Epoch 2/10
8/8 [==============================] - 3s 372ms/step - loss: 2.1541 - accuracy: 0.2465 - val_loss: 2.2977 - val_accuracy: 0.2001

Epoch 00002: val_loss improved from 2.31123 to 2.29773, saving model to m_pol_mnist_layerAdjust.h5
Epoch 3/10
8/8 [==============================] - 3s 368ms/step - loss: 1.9225 - accuracy: 0.3692 - val_loss: 2.3088 - val_accuracy: 0.2602

Epoch 00003: val_loss did not improve from 2.29773
Epoch 4/10
8/8 [==============================] - 3s 371ms/step - loss: 1.6006 - accuracy: 0.5016 - val_loss: 2.3507 - val_accuracy: 0.3050

Epoch 00004: val_loss did not improve from 2.29773
Epoch 5/10
8/8 [==============================] - 3s 394ms/step - loss: 1.1882 - accuracy: 0.6492 - val_loss: 2.3547 - val_accuracy: 0.3262
